In [1]:
import sys
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')

import itertools
import re

import glob, os
import pandas as pd
import json
import spacy
import nltk
import numpy as np
import json
#import smogn
import seaborn as sns
import pickle

import joblib
from joblib import Parallel, delayed

from bs4 import BeautifulSoup
from pysbd.utils import PySBDFactory
import math

from sumeval.metrics.rouge import RougeCalculator
rouge = RougeCalculator(stopwords=True, lang="en")
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from timeit import default_timer as timer 
#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import RandomUnderSampler
#from imblearn.pipeline import Pipeline
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
#from tensorflow.keras.models import model_from_json

#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('punkt')
#nltk.download('stopwords')

#!python -m spacy download en_core_web_sm
nlp_sm = spacy.load('en_core_web_sm')

#import en_core_web_sm
#nlp_md = en_core_web_sm.load()

#import en_core_web_md
#nlp_md = en_core_web_md.load()
#!python -m spacy download en_core_web_md
nlp_md = spacy.load('en_core_web_md')

import warnings
warnings.filterwarnings("ignore")

path_base = "/scratch/cinthiasouza/mv-text-summarizer"
path_to_read="/scratch/cinthiasouza/mv-text-summarizer/result/{}/{}_*.csv"

In [ ]:
#!/scratch/cinthiasouza/anaconda3/envs/mvsumm/lib python3.8

In [2]:
%load_ext autoreload
%autoreload 2

from src import preprocess
from src import extract_features
from src import tokenizer
from src import create_features_df
from src import transform_data
from src import loader
from src import utils
#from src import ensemble_tree_models
from src import tunning_hyperparametrs as th
from src import mlp_classifier
#from src import summarization
#from src import normalization
from src import ensemble_tree_models as classifiers
from src import utils_classification as utils_clf
from src import evaluate_classifiers as ev
from src import prepare_data
from src import display_results as dr
import joblib
from joblib import Parallel, delayed
#from tensorflow.keras.utils import to_categorical
#from src import pipeline_extract_features as pef

# Load Features

In [ ]:
"""import random

features_intro, scores_intro = loader.read_features(path="../result/introduction/features_*.csv")
features_mat, scores_mat = loader.read_features(path="../result/materials/features_*.csv")
features_conc, scores_conc = loader.read_features(path="../result/conclusion/features_*.csv")

intro = pd.unique(features_intro['articles'])
mat = pd.unique(features_mat['articles'])
conc = pd.unique(features_conc['articles'])

comuns = list((set(intro) & set(mat)) & set(conc))

valid_len = int(len(comuns)*0.2)
summ_items = random.sample(comuns, valid_len)

df = pd.DataFrame({'summ': summ_items})
df.to_csv("indices_summ.csv", index=False)"""

In [ ]:
def main(path_to_read, name_csv="features", format_dataset=True, verbose=True):
    
    columns_name = ['text_rank', 'lex_rank', 'count_one_gram', 'count_two_gram',
       'count_three_gram', 'count_article_keywords',
       'tf-isf', 'position_score', 'paragraph_score',
       'length_score', 'pos_score', 'ner_score', 'dist_centroid']

    sections=['introduction', 'materials', 'conclusion']
    #sections=[ 'introduction']

    if format_dataset:
        if verbose:
            print("Preparando dataset para os classificadores")
        dataset = prepare_data.main_create_dataset(columns_name, sections, path_to_read, name_csv)
        #utils.,save_json(dataset, name='dataset', path=path)
    else:
        if verbose:
            print("Carregando dataset")
        dataset = utils.load_json(name='dataset', path=path)
    
    if verbose:
        print("Treinamento dos modelos")
        
    with open('dataset5_{}.pkl'.format(name_csv), 'wb') as fp:
        pickle.dump(dataset, fp, protocol=pickle.HIGHEST_PROTOCOL)
        
    return dataset
    
    '''models = ensemble_tree_models.create_models(dataset, sections, name_models=['knn', 'gb', 'rf', 'ab'])
    predictions, results = ev.create_reports(models, dataset, columns_name, verbose=False)
    
    #utils.save_json(predictions, name='prediction', path=path)
    #utils.save_results(results, path=path)
    
    parameters = {'introduction': [0.2, 100, 64],
             'materials': [0.2, 100, 64],
             'conclusion':[0.2, 100, 64],
             'concat': [0.2, 100, 64]}

    models_nn = mlp_classifiers.main_train_nn(dataset, sections, parameters, train=True, verbose=False)
    predictions, results =  mlp_classifiers.eval_nn(dataset, sections)'''
    

In [ ]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument('--path', "-p",required=True)
    parser.add_argument('--format_dataset', "-f",required=True)
     
    args = parser.parse_args()

    columns_name = ['text_rank', 'lex_rank', 'count_one_gram',
        'count_article_keywords',
       'tf-isf', 'position_score', 'paragraph_score',
       'length_score', 'pos_score', 'ner_score', 'dist_centroid']

    sections=['introduction', 'materials', 'conclusion', 'concat']

In [4]:
dataset, train, test  = main(verbose=True, path_to_read=path_to_read, name_csv='features')

Preparando dataset para os classificadores
/scratch/cinthiasouza/mv-text-summarizer/result/introduction/features_*.csv
/scratch/cinthiasouza/mv-text-summarizer/result/materials/features_*.csv
/scratch/cinthiasouza/mv-text-summarizer/result/conclusion/features_*.csv
Treinamento dos modelos


In [23]:
test.to_csv("test5.csv", index=False)

models <-{0:0.9, 1:0.1} dataset3 [intro, mat, conc]

models_v2 <-{0:0.95, 1:0.05} dataset3 [intro, mat, conc]

models_v3 <-{0:0.95, 1:0.05} dataset4 [intro, mat, conc]

models_v4 <-{0:0.95, 1:0.05} dataset4 [concat]

models_v5 <-{0:1, 1:10} dataset4 [intro, mat, conc]

In [3]:
with open('dataset5_{}.pkl'.format('features'), 'rb') as fp:
    dataset = pickle.load(fp)

columns_name = ['text_rank', 'lex_rank', 'count_one_gram', 'count_two_gram',
       'count_three_gram', 'count_article_keywords', 'tf-isf', 'pos_score', 'ner_score']

sections=['introduction', 'materials', 'conclusion']

folder_to_save = 'models_v5'

path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/{}".format(folder_to_save)

## Randomized Search -  CV 5 Folds

In [4]:
class_weight_intro = {0:7, 1:1}
class_weight_mat = {0:6, 1:1}
class_weight_conc = {0:12, 1:1}

In [5]:
parameters_tunning = {
    
'introduction': {'rf': {'n_estimators': [100, 150, 300],
              'min_samples_leaf':[100, 500, 200],
              'min_samples_split':[200, 1000, 30000],
              'max_depth':[10, 20, 30, 50],
              'class_weight': class_weight_intro
              },
              'gb': {'n_estimators': [50, 100, 300],
              'min_samples_leaf':[10, 50, 100, 200, 1000],
              'min_samples_split':[100, 500, 750, 1000],
              'max_depth':[10, 20, 30, 40, 50]        
              },
              'cb': {'iterations': [100, 150, 300],
            'learning_rate': [0.01, 0.001, 0.0001],
            'depth': [10, 20, 30],
            'class_weights':class_weight_intro,
            'min_data_in_leaf':[500, 1000, 2000]},
              'knn':  {'n_neighbors': [5, 10, 20, 50]},
              'ab': {'n_estimators': [50,  100, 200]},
              'svm': {'kernel': ('poly', 'rbf'), 'degree':[2, 3, 4], 'class_weight': class_weight_intro}
                 },

"materials": {'rf': {'n_estimators': [100, 150, 300],
              'min_samples_leaf':[100, 500, 200],
              'min_samples_split':[200, 1000, 30000],
              'max_depth':[10, 20, 30, 50],
              'class_weight': class_weight_mat
              },
              'gb': {'n_estimators': [50, 100, 150, 200, 300],
              'min_samples_leaf':[5, 10, 100],
              'min_samples_split':[10, 20],
              'max_depth':[20, 10, 50, 100]
              },
              'cb': {'iterations': [100, 150, 300],
            'learning_rate': [0.01, 0.001, 0.0001],
            'depth': [10, 20, 30],
            'class_weights':class_weight_mat,
            'min_data_in_leaf':[500, 1000, 2000]},
              'knn':  {'n_neighbors': [5, 10, 20, 50]},
              'ab': {'n_estimators': [50,  100, 200]},
              'svm': {'kernel': ('poly', 'rbf'), 'degree':[2, 3, 4], 'class_weight': class_weight_mat}
                 },

'conclusion': {'rf': {'n_estimators': [100, 150, 300],
              'min_samples_leaf':[100, 500, 200],
              'min_samples_split':[200, 1000, 30000],
              'max_depth':[10, 20, 30, 50],
              'class_weight': class_weight_conc
              },
               'gb': {'n_estimators': [50, 100, 150, 200, 300],
              'min_samples_leaf':[5, 10, 100],
              'min_samples_split':[10, 20],
              'max_depth':[20, 50, 100]},
               'cb': {'iterations': [100, 150, 300],
            'learning_rate': [0.01, 0.001, 0.0001],
            'depth': [10, 20, 30],
            'class_weights':class_weight_conc,
            'min_data_in_leaf':[500, 1000, 2000]},
              'knn':  {'n_neighbors': [5, 10, 20, 50]},
              'ab': {'n_estimators': [50,  100, 200]},
              'svm': {'kernel': ('poly', 'rbf'), 'degree':[2, 3, 4], 'class_weight': class_weight_conc}
                 }
    }

In [ ]:
models_tunning = {'gb': GradientBoostingClassifier(), 
                          'rf': RandomForestClassifier(n_jobs=-1, class_weight={0:1, 1:3}),
                          'ab': AdaBoostClassifier(DecisionTreeClassifier(class_weight={0:1, 1: 3})),
                          'knn': KNeighborsClassifier(n_jobs=-1),
                          'cb': CatBoostClassifier(class_weights={0:1, 1: 3}),
                          'svm': SVC(class_weight={0:1, 1: 3})}

In [ ]:
th.pipeline_tunning(
         dataset, {'rf': RandomForestClassifier(n_jobs=-1, class_weight=class_weight)},
        sections, parameters_tunning, path_to_save)

In [ ]:
th.pipeline_tunning(
         dataset, {'gb': GradientBoostingClassifier()},
        sections, parameters_tunning, path_to_save)

In [14]:
th.pipeline_tunning(
         dataset, {'ab': AdaBoostClassifier()},
        sections, parameters_tunning, path_to_save)

In [13]:
th.pipeline_tunning(
         dataset, {'knn': KNeighborsClassifier()},
        sections, parameters_tunning, path_to_save)

In [6]:
def load_rs_model(name_model, sections, path_to_read):
    
    rs_models = {}
    
    for section in sections:
        
        rs_models[section] = joblib.load('{}/search_{}_{}.pkl'.format(path_to_read, name_model, section))
        
    return rs_models

In [7]:
def  get_scores(name_models, sections, path_to_read):

    results = {}
    
    for name_model in name_models:
        
        rs_models = load_rs_model(name_model, sections, path_to_read)
    
        aux  = { 'score_{}'.format(section): rs_models[section].best_score_ for section in sections}
        results[name_model] = aux
        
    return results

In [ ]:
sections=['introduction', 'materials', 'conclusion']
name_models = ['knn', 'ab', 'gb']

results = get_scores(name_models, sections, path_to_save)

## Results K-Fold Cross Validation

In [ ]:
cv_results = pd.DataFrame(results).T
cv_results

## Hyperparameters

In [16]:
def get_models_rs(name_models, sections, path_to_read, verbose=False):

    searches = {}

    for name_model in name_models:

        aux = {}

        for section in sections:

            aux[section] = joblib.load('{}/search_{}_{}.pkl'.format(path_to_read, name_model, section))

            if verbose: 
                print(aux[section].best_params_)

        searches[name_model] = aux

    return searches


In [17]:
def get_dict_parameters(searches, name_models, sections):

    parameters = {}

    for name_model in name_models:

        aux = {}

        for section in sections:

            aux[section] = searches[name_model][section].best_params_

        parameters[name_model] = aux
    
    return parameters

In [11]:
sections=['introduction', 'materials', 'conclusion']
name_models = ['rf', 'cb']

In [36]:
searches = get_models_rs(name_models, sections, path_to_save)

In [37]:
parameters = get_dict_parameters(searches, name_models, sections)

In [70]:
class_weight_intro = {0:3, 1:1}
class_weight_mat = {0:3, 1:1}
class_weight_conc = {0:3, 1:1}

parameters = {'rf': {'introduction': {'n_estimators': 300,
   'min_samples_split': 200,
   'min_samples_leaf': 10,
   'max_depth': 50,
   'class_weight': class_weight_intro},
  'materials': {'n_estimators': 300,
   'min_samples_split': 30000,
   'min_samples_leaf': 10,
   'max_depth': 50,
   'class_weight': class_weight_mat},
  'conclusion': {'n_estimators': 150,
   'min_samples_split': 1000,
   'min_samples_leaf': 10,
   'max_depth': 50,
   'class_weight': class_weight_conc}},
 'cb': {'introduction': {'min_data_in_leaf': 10,
   'learning_rate': 0.01,
   'iterations': 100,
   'depth': 10,
   'class_weights': class_weight_intro},
  'materials': {'min_data_in_leaf': 10,
   'learning_rate': 0.001,
   'iterations': 100,
   'depth': 10,
   'class_weights': class_weight_mat},
  'conclusion': {'min_data_in_leaf': 10,
   'learning_rate': 0.0001,
   'iterations': 150,
   'depth': 10,
   'class_weights': class_weight_conc}}}

In [6]:
class_weight_intro = {0:7, 1:1}
class_weight_mat = {0:6, 1:1}
class_weight_conc = {0:12, 1:1}

parameters = {'rf': {'introduction': {'n_estimators': 500,
   'min_samples_split': 200,
   'min_samples_leaf': 500,
   'max_depth': 30,
   'class_weight': class_weight_intro},
  'materials': {'n_estimators': 500,
   'min_samples_split': 1000,
   'min_samples_leaf': 500,
   'max_depth': 10,
   'class_weight': class_weight_mat},
  'conclusion': {'n_estimators': 500,
   'min_samples_split': 1000,
   'min_samples_leaf': 500,
   'max_depth': 30,
   'class_weight': class_weight_conc}},
 'cb': {'introduction': {'min_data_in_leaf': 500,
   'learning_rate': 0.01,
   'iterations': 100,
   'depth': 10,
   'class_weights': class_weight_intro},
  'materials': {'min_data_in_leaf': 500,
   'learning_rate': 0.001,
   'iterations': 100,
   'depth': 10,
   'class_weights': class_weight_mat},
  'conclusion': {'min_data_in_leaf': 500,
   'learning_rate': 0.01,
   'iterations': 200,
   'depth': 10,
   'class_weights': class_weight_conc}}}

## Train Test

In [7]:
classifiers.pipeline_classifiers(dataset, parameters, sections, ['rf'], folder_to_save)

{'introduction': {'rf': RandomForestClassifier(class_weight={0: 7, 1: 1}, max_depth=30,
                         min_samples_leaf=500, min_samples_split=200,
                         n_estimators=500, n_jobs=-1)},
 'materials': {'rf': RandomForestClassifier(class_weight={0: 6, 1: 1}, max_depth=10,
                         min_samples_leaf=500, min_samples_split=1000,
                         n_estimators=500, n_jobs=-1)},
 'conclusion': {'rf': RandomForestClassifier(class_weight={0: 12, 1: 1}, max_depth=30,
                         min_samples_leaf=500, min_samples_split=1000,
                         n_estimators=500, n_jobs=-1)}}

In [8]:
classifiers.pipeline_classifiers(dataset, parameters, sections, ['cb'], folder_to_save)

{'introduction': {'cb': <catboost.core.CatBoostClassifier at 0x7f45168b6490>},
 'materials': {'cb': <catboost.core.CatBoostClassifier at 0x7f45168b6520>},
 'conclusion': {'cb': <catboost.core.CatBoostClassifier at 0x7f45168b6280>}}

# Predict

In [ ]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision

In [ ]:
def load_keras_model(section, path_to_save):

    json_file = open('{}/mlp_{}.json'.format(path_to_save, section), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights('{}/mlp_{}.h5'.format(path_to_save, section))
    print("Loaded model from disk")
    
    loaded_model.compile(loss='categorical_crossentropy', optimizer=Adam(
                learning_rate=0.001), metrics=[Precision()])
    
    return loaded_model

In [ ]:
def load_classifiers(sections, path_to_read, name_models):

    models = {}
    
    for section in sections:
        
        aux = {}
        
        for name_model in name_models:
            if name_model != 'mlp':
                aux[name_model] = joblib.load('{}/{}_{}.pkl'.format(path_to_read, name_model,  section))
            elif name_model == 'mlp':
                aux[name_model] = load_keras_model(section, path_to_read)
            
        models[section] = aux
        
    return models

In [ ]:
sections=['introduction', 'materials', 'conclusion']
name_models = ['rf', 'cb', 'mlp']

models = load_classifiers(sections, path_to_save, name_models)

In [ ]:
predictions, predictions_proba, results = ev.create_reports(models, dataset, columns_name, name_models, verbose=False)

In [ ]:
results['introduction']['rf']

In [ ]:
results['introduction']['rf']

In [ ]:
results['introduction']['rf']

In [ ]:
results['introduction']['rf']

In [ ]:
results['materials']['rf']

In [ ]:
results['materials']['rf']

In [ ]:
results['conclusion']['rf']

In [ ]:
results['materials']['cb']

In [ ]:
dr.display_results4(results, section='introduction')

In [ ]:
dr.display_results4(results, section='materials')

In [ ]:
dr.display_results4(results, section='conclusion')

# Matthews Correlation

In [ ]:
df = ev.matthews(sections, dataset, predictions, name_models)
df

# ROC

In [ ]:
ev.roc_curve(sections, dataset, models)

In [ ]:
import numpy as np
from sklearn.ensemble import VotingClassifier

from sklearn.tree import DecisionTreeClassifier

n_estimators = 300
estimators = []

for i in range(300):
    
    rf =  DecisionTreeClassifier(
        max_depth=10, splitter='best', min_samples_split=200,
        min_samples_leaf= 100, class_weight={0:0.98,1:0.02})
    
    estimators.append(('rf_' + str(i), rf))


model = VotingClassifier(estimators, voting='hard')
model = model.fit(dataset['introduction'][0], dataset['introduction'][2])
predictions = model.predict(dataset['introduction'][1])


In [ ]:
# pytorch mlp for multiclass classification
from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [ ]:
# pytorch mlp for multiclass classification
from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
 
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
 
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 3)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Softmax(dim=1)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # output layer
        X = self.hidden3(X)
        X = self.act3(X)
        return X
 
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl
 
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(500):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
 
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # convert to class labels
        yhat = argmax(yhat, axis=1)
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc
 
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat
 
# prepare the data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(4)
# train the model
train_model(train_dl, model)
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)
# make a single prediction
row = [5.1,3.5,1.4,0.2]
yhat = predict(row, model)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))


In [ ]:
from src import mlp_classifier

In [ ]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
#from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import model_from_json

#from keras.callbacks import LearningRateScheduler
#from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report

import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical

In [ ]:
parameters_mlp = { 
    'introduction':  [ 0.2, 100, 64],
    'materials':     [ 0.2, 100, 64],
    'conclusion':  [ 0.2, 100, 64]}

In [ ]:
models = mlp_classifier.main_train_nn(dataset, sections, parameters_mlp, train=True, verbose=True)

In [ ]:
section = 'introduction'

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=13, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(
            learning_rate=0.001), metrics=['accuracy', keras.metrics.AUC()])

In [ ]:
X = dataset[section][0]
y = dataset[section][2]

In [ ]:
one_hot_label = to_categorical(y)

In [ ]:
test_size, epochs, batch_size =parameters_mlp.get(section)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, one_hot_label, stratify=one_hot_label, test_size=test_size)

In [ ]:
len(X_train)

In [ ]:
len(y_train)

In [ ]:
model.fit(
        np.array(X_train), np.array(y_train), validation_data=(X_valid,y_valid), epochs=epochs,
         batch_size=batch_size, class_weight={0:0.95, 1:0.05}, verbose=True)